In [21]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import cross_val_score


In [2]:
list=["biology","cooking","crypto","diy","robotics","travel"]

Make a new dataframe called training_set consisting of the training set data.

In [3]:
training_list=[]
for x in list:
    path = 'stackexchange/{}.csv'.format(str(x))
    training_list.append(pd.read_csv(path,usecols=['title','content','tags'],index_col=0))
training_set = pd.concat(training_list, axis=0 , join='outer')
training_set.dropna()
training_set= training_set.reset_index()
training_set.head(10)

,title,content,tags
0,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...
1,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry
2,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology
3,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture
4,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons
5,How can I avoid digesting protein-bound DNA?,<p>I'm interested in sequencing and analyzing ...,dna biochemistry molecular-biology
6,Under what conditions do dendritic spines form?,<p>I'm looking for resources or any informatio...,neuroscience synapses
7,How should I ship plasmids?,<p>I shipped 10 µL of my vector miniprep to a ...,plasmids
8,What is the reason behind choosing the reporte...,<p>I noticed within example experiments in cla...,molecular-genetics gene-expression experimenta...
9,How many times did endosymbiosis occur?,"<p>According to the endosymbiont theory, mitoc...",evolution mitochondria chloroplasts


In [4]:
training_set.shape

(87000, 3)

In [5]:
training_set['content'] = training_set['content'].map(lambda x: x.lstrip('<p>').strip('<p').rstrip('<p>'))

In [6]:
training_set.ix[[1111]]

,title,content,tags
1111,Are there any effects of elevated Cysteine lev...,I'm looking at this diagram of homocysteine me...,human-biology amino-acids vitamins dreaming


In [12]:
training_set.ix[[55555]]

,title,content,tags
55555,Repair L-shaped notch in drywall,I cut some L-shaped notches in the corner betw...,electrical drywall patching-drywall


In [14]:
training_set.data

AttributeError: 'DataFrame' object has no attribute 'data'

In [38]:
print(training_set['tags'].values)

['ribosome binding-sites translation synthetic-biology' 'rna biochemistry'
 'immunology cell-biology hematology' ...,
 'customs-and-immigration officials registration macedonia' 'visas austria'
 'untagged']


In [30]:
test = zip(training_set['title'].values,training_set['content'].values,training_set['tags'].values)

In [31]:
test